In [ ]:
import pandas as pd
import mysql.connector

In [ ]:
db_config = {
    "host": "localhost",
    "user": "john",
    "password": "blueharb0r!",
    "database": "blueharbor"
}

conn = mysql.connector.connect(**db_config)

In [ ]:
query = """
SELECT s.shipment_id, s.origin_country, s.destination_country, s.product_id, s.declared_value, s.weight_kg, s.shipment_date, s.incoterm, i.invoice_id, i.invoice_value, i.vat_amount, i.customs_amount, i.tax_flag 
FROM shipments s
JOIN invoices i ON s.shipment_id = i.shipment_id;

"""

In [ ]:
df = pd.read_sql(query,conn)
conn.close()

In [ ]:
print(df.head())
print(f"Total rows: {len(df)}")

A brief view of the indirect tax landscape:

In [ ]:
#total VAT collected by dest country
vat_per_country = df.groupby('destination_country')['vat_amount'].sum().sort_values(ascending=False)
print(vat_per_country)

#total custom collected by dest country
customs_per_country = df.groupby('destination_country')['customs_amount'].sum()
print(customs_per_country)

#shipments per product
ship_per_product = df.groupby('product_id')['shipment_id'].count()
print(ship_per_product)

#count missing vat invoices
count_missing_vat = df[df['tax_flag'] == 'MISSING_VAT'].shape[0]
print(f"{count_missing_vat} invoices missing VAT")

Complete view of each shipment, joining tables together.

In [ ]:
import pandas as pd
import mysql.connector

In [ ]:
db_config = {
    "host": "localhost",
    "user": "john",
    "password": "blueharb0r!",
    "database": "blueharbor"
}

conn = mysql.connector.connect(**db_config)

In [ ]:
query2 = """
select
    s.shipment_id, s.origin_country, s.destination_country, s.declared_value, s.weight_kg, s.incoterm, s.shipment_date,
    p.product_name, p.product_category,
    t.vat_rate, t.customs_duty_rate,
    i.invoice_id, i.invoice_value, i.vat_amount, i.customs_amount, i.tax_flag, i.invoice_date
from shipments s
join invoices i on s.shipment_id = i.shipment_id
join products p on s.product_id = p.product_id
left join tax_rates t on s.destination_country = t.country;
"""

In [ ]:
df = pd.read_sql(query2, conn)
conn.close()

In [ ]:
print(df.head())
print(f"{len(df)} rows and {df.shape[1]} columns")